# Evaluating circuit-level performance

# Zoned archetectures, Gemini noise models
Describe our noise model (cross reference the Gemini benchmarking paper)

“Hello Gemini” level hardware description. Native gate sets; parallelism; atom moving; loss.

Describe our strategy for noise modeling: Pauli channels plus atom loss

Canonical circuit we will use for this example: Trotterization of 1d Ising.

In [ ]:
import bloqade
import bloqade.squin
import kirin

@bloqade.squin.kernel
def bloqade_trotter(qubits:list[bloqade.squin.qubit], steps:int, dt:float = 0.01, J:float = 1, h:float = 1):
    """
    Main function that runs the Trotter circuit for a given number of steps
    """
    for _ in range(steps):
        zz = bloqade.squin.op.zzpow(dt * J)
        x = bloqade.squin.op.xpow(dt * h)
        for i in range(0, len(qubits) - 1, 2):
            bloqade.squin.op.apply(zz, qubits[i], qubits[i + 1])
        for i in range(1, len(qubits) - 1, 2):
            bloqade.squin.op.apply(zz, qubits[i], qubits[i + 1])
        for i in range(0, len(qubits)):
            bloqade.squin.op.apply(x, qubits[i])

def trotter_builder(nqubits:int,
                    steps:int = 100,
                    dt:float = 0.01,
                    J:float = 1,
                    h:float = 1)-> kirin.ir.Kernel:
    """
    Builder function to create a Trotter circuit with the specified parameters.
    
    Inputs:
        nqubits: Number of qubits in the circuit.
        steps: Number of Trotter steps to perform.
        dt: Time step for the Trotter evolution.
        J: Coupling strength for the ZZ interaction.
        h: Strength of the X field.
    Returns:
        A kernel function with no inputs and returns the qubits after the Trotter evolution.
    """
    def main():
        qubits = bloqade.squin.qubit.create(nqubits)
        bloqade_trotter(qubits, steps, dt, J, h)
        return qubits
    
    # Enforce the CZ + XYphase structure with a compiler pass
    main2 = bloqade.squin.passes.CZGateSet()(main)
    
    return main2

Now that we have specified the circuit, we can start doing analysis...

Simplest analysis: counting the number of gates and estimating a utility function

## Log fidelity analysis
Count the number of gates; log fidelity is simply a weighted sum/dot product.

In [ ]:
# Only works with no mid-circuit feed-forward? Optionally, run statevector evolution
# and sample from the distribution to get a monte carlo estimate of the counting.
noise_interpreter = bloqade.circuit.interpreters.noise_counting()

report = noise_interpreter.run(trotter_builder(10, steps=10, dt=0.01, J=1, h=1))
report.print()
# Number of qubits:        10
# Number of 2 qubit gates: 90
# Number of 1 qubit gates: 100
# Number of touches:       50
# Number of moves:         10

error_model = bloqade.circuit.GeminiErrorModel(scale = 0.7)
log_fidelity:float = report.log_fidelity(error_model)
print(f"Log fidelity: {log_fidelity:.3f}")
# Log fidelity: -1.234

## Noise annotation in Cirq

Reproduce and reference [cirq_tools](https://github.com/QuEraComputing/cirq_tools/)

## Noise annotation in Bloqade

In [ ]:
raise NotImplementedError("We cannot do noise simulation with bloqade")